In [1]:
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

import cirq
from cirq.contrib.svg import SVGCircuit

In [2]:
import networkx as nx
SYC23_GRAPH = nx.from_edgelist([
    ((3, 2), (4, 2)), ((4, 1), (5, 1)), ((4, 2), (4, 1)), 
    ((4, 2), (4, 3)), ((4, 2), (5, 2)), ((4, 3), (5, 3)), 
    ((5, 1), (5, 0)), ((5, 1), (5, 2)), ((5, 1), (6, 1)), 
    ((5, 2), (5, 3)), ((5, 2), (6, 2)), ((5, 3), (5, 4)), 
    ((5, 3), (6, 3)), ((5, 4), (6, 4)), ((6, 1), (6, 2)), 
    ((6, 2), (6, 3)), ((6, 2), (7, 2)), ((6, 3), (6, 4)), 
    ((6, 3), (7, 3)), ((6, 4), (6, 5)), ((6, 4), (7, 4)), 
    ((6, 5), (7, 5)), ((7, 2), (7, 3)), ((7, 3), (7, 4)), 
    ((7, 3), (8, 3)), ((7, 4), (7, 5)), ((7, 4), (8, 4)), 
    ((7, 5), (7, 6)), ((7, 5), (8, 5)), ((8, 3), (8, 4)), 
    ((8, 4), (8, 5)), ((8, 4), (9, 4)), 
])

In [3]:
from recirq.otoc.loschmidt import *

In [4]:
for topo in get_all_diagonal_rect_topologies():
    print(topo, topo.n_qubits)

DiagonalRectangleTopology(width=1, height=1) 5
DiagonalRectangleTopology(width=1, height=2) 8
DiagonalRectangleTopology(width=1, height=3) 11
DiagonalRectangleTopology(width=1, height=4) 14
DiagonalRectangleTopology(width=2, height=2) 13
DiagonalRectangleTopology(width=2, height=3) 18
DiagonalRectangleTopology(width=2, height=4) 23
DiagonalRectangleTopology(width=3, height=3) 25
DiagonalRectangleTopology(width=3, height=4) 32
DiagonalRectangleTopology(width=4, height=4) 41


In [5]:
np.arange(1, 10, 2)

array([1, 3, 5, 7, 9])

In [6]:
pg = get_all_diagonal_rect_executables(min_side_length=1, max_side_length=3)

In [7]:
def recurse_pg_html(pg:Union[Program, ProgramGroup], depth=0):
    if isinstance(pg, ProgramGroup):
        s = ' '*depth + f'<li>Group {pg.info}</span><ul class="nested">\n'
    
        for child in pg.programs:
            s += recurse_pg_html(child, depth=depth+1)
        s += ' '*depth + '</ul></li>\n'
        return s
    
    s = ' ' * depth + '<li>' + str(pg) + '</li>\n'
    return s
        
s = recurse_pg_html(pg)
print(s)

<li>Group (('name', 'recirq.otoc.loschmidt'),)</span><ul class="nested">
 <li>Group (('topology', DiagonalRectangleTopology(width=1, height=1)),)</span><ul class="nested">
  <li>Group (('macrocyle_depth', 1),)</span><ul class="nested">
   <li>Program(info=(('instance_i', 0),))</li>
   <li>Program(info=(('instance_i', 1),))</li>
   <li>Program(info=(('instance_i', 2),))</li>
   <li>Program(info=(('instance_i', 3),))</li>
   <li>Program(info=(('instance_i', 4),))</li>
   <li>Program(info=(('instance_i', 5),))</li>
   <li>Program(info=(('instance_i', 6),))</li>
   <li>Program(info=(('instance_i', 7),))</li>
   <li>Program(info=(('instance_i', 8),))</li>
   <li>Program(info=(('instance_i', 9),))</li>
  </ul></li>
  <li>Group (('macrocyle_depth', 3),)</span><ul class="nested">
   <li>Program(info=(('instance_i', 0),))</li>
   <li>Program(info=(('instance_i', 1),))</li>
   <li>Program(info=(('instance_i', 2),))</li>
   <li>Program(info=(('instance_i', 3),))</li>
   <li>Program(info=(('instan

In [8]:
class pgview:
    def __init__(self, pg):
        self.pg = pg
        
    def _repr_html_(self):
        return '<ul>\n' + recurse_pg_html(self.pg) + '</ul>\n'
        
pgview(pg)

In [9]:
def recurse_pg_flatten(pg:Union[Program, ProgramGroup], flat_list, info, parents):
    new_info = dict(info)
    pg_info = dict(pg.info)
    for k in pg_info:
        if k in new_info:
            raise ValueError("Key already exists")
        new_info[k] = pg_info[k]
    new_info=tuple(new_info.items())
    
    if isinstance(pg, ProgramGroup):
        for child in pg.programs:
            recurse_pg_flatten(child, flat_list=flat_list, info=new_info, parents=(pg,) + parents)
    else:
        flat_list.append((pg, new_info, parents))
        
flat_list = []
recurse_pg_flatten(pg, flat_list, {}, ())
for program, info, parents in flat_list:
    print(program, parents)
    
    
import pandas as pd
pd.DataFrame([info for program, info, parents in flat_list])

Program(info=(('instance_i', 0),)) (ProgramGroup(info=(('macrocyle_depth', 1),)) at 140586225096448, ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=1)),)) at 140586224942336, ProgramGroup(info=(('name', 'recirq.otoc.loschmidt'),)) at 140586224985472)
Program(info=(('instance_i', 1),)) (ProgramGroup(info=(('macrocyle_depth', 1),)) at 140586225096448, ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=1)),)) at 140586224942336, ProgramGroup(info=(('name', 'recirq.otoc.loschmidt'),)) at 140586224985472)
Program(info=(('instance_i', 2),)) (ProgramGroup(info=(('macrocyle_depth', 1),)) at 140586225096448, ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=1)),)) at 140586224942336, ProgramGroup(info=(('name', 'recirq.otoc.loschmidt'),)) at 140586224985472)
Program(info=(('instance_i', 3),)) (ProgramGroup(info=(('macrocyle_depth', 1),)) at 140586225096448, ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1

,0,1,2,3
0,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=1, ...","(macrocyle_depth, 1)","(instance_i, 0)"
1,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=1, ...","(macrocyle_depth, 1)","(instance_i, 1)"
2,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=1, ...","(macrocyle_depth, 1)","(instance_i, 2)"
3,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=1, ...","(macrocyle_depth, 1)","(instance_i, 3)"
4,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=1, ...","(macrocyle_depth, 1)","(instance_i, 4)"
...,...,...,...,...
295,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=3, ...","(macrocyle_depth, 9)","(instance_i, 5)"
296,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=3, ...","(macrocyle_depth, 9)","(instance_i, 6)"
297,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=3, ...","(macrocyle_depth, 9)","(instance_i, 7)"
298,"(name, recirq.otoc.loschmidt)","(topology, DiagonalRectangleTopology(width=3, ...","(macrocyle_depth, 9)","(instance_i, 8)"


In [10]:
hash(program.circuit.freeze())

-174602187169148696

In [11]:
import dataclasses
from typing import List
from collections import defaultdict

In [12]:
directories = set()
children = defaultdict(set)

for program, info, parents in flat_list:
    result = str(program.uuid)
    # TODO: order
    children[parents[0]].add(result)
    
    for p1, p2 in zip(parents, parents[1:]):
        directories.add((p1, p2))
        children[p2].add(p1)
    
    
#directories
children

defaultdict(set,
            {ProgramGroup(info=(('macrocyle_depth', 1),)) at 140586225096448: {'07c36dbc-330d-4d78-ad71-12f3e67b4fbc',
              '1a843d20-36bc-4aa2-baa3-c28b783fc70d',
              '508e3729-ed72-40f4-b526-fe9c30914a31',
              '512c9730-e495-4bba-9333-7a5def018b74',
              '61a6aed0-1a7f-409c-b516-c0bf08fd6436',
              '6462ad61-a8bc-483c-bb6d-40c8bafcbb17',
              '9fd32d54-d4ca-455f-bbd0-ea22eb03767b',
              'aceb0511-2f1e-4ae7-b4a2-a76442aeae6b',
              'c84cf3bc-8500-44e7-852b-4e3616525c2f',
              'fe13f45a-c9ab-45db-b350-af5c049d7cca'},
             ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=1)),)) at 140586224942336: {ProgramGroup(info=(('macrocyle_depth', 1),)) at 140586225096448,
              ProgramGroup(info=(('macrocyle_depth', 3),)) at 140586225824096,
              ProgramGroup(info=(('macrocyle_depth', 5),)) at 140586226947168,
              ProgramGroup(info=(('macr

In [13]:
children[pg]

{ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=1)),)) at 140586224942336,
 ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=2)),)) at 140586234498544,
 ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=1, height=3)),)) at 140586247011008,
 ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=2, height=2)),)) at 140586263985168,
 ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=2, height=3)),)) at 140585745414080,
 ProgramGroup(info=(('topology', DiagonalRectangleTopology(width=3, height=3)),)) at 140585772903728}

In [14]:
assert False

AssertionError: 

In [ ]:
from recirq.named_topologies import DiagonalRectangleTopology, get_placements
rr = DiagonalRectangleTopology(1, 1).graph
subgs = get_placements(SYC23_GRAPH, rr, plt=plt)

In [ ]:
subg = subgs[0]
sorted(subg.nodes)

In [ ]:
from recirq.otoc.loschmidt import create_diagonal_rectangle_loschmidt_echo_circuit

circuit = create_diagonal_rectangle_loschmidt_echo_circuit(
    qubits = [cirq.GridQubit(r, c) for r, c in sorted(subg.nodes)],
    cycles=4,
    seed=52,
)
SVGCircuit(circuit)

In [ ]:
cirq.final_state_vector(circuit).round(5)

In [ ]:
len(circuit)

In [ ]:
c2 = cirq.Circuit(circuit.all_operations())
print(len(circuit), len(c2))
SVGCircuit(c2)

In [ ]:
import cirq_google as cg
sampler = cg.get_engine_sampler('weber', 'sqrt_iswap')
sampler = cirq.Simulator()

In [ ]:
sorted(circuit.all_qubits())

In [ ]:
def get_success(r):
    return np.all(r.measurements['z'] == np.array([0] * len(circuit.all_qubits())), axis=1).astype(int)

In [ ]:
r1 = sampler.run(circuit + cirq.measure(*sorted(circuit.all_qubits()), key='z'), repetitions=50_000)
get_success(r1)

In [ ]:
r2 = sampler.run(c2      + cirq.measure(*sorted(circuit.all_qubits()), key='z'), repetitions=50_000)
get_success(r2)

In [ ]:
np.mean(get_success(r1))

In [ ]:
np.mean(get_success(r2))